In [1]:
!pip3 install -r requirements.txt

from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import nn
import torch
import csv
from random import choice
import wandb
import plotly.express as px
import numpy as np
import umap.umap_ as umap
import os


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readth

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shayaan-absar. Use `wandb login --relogin` to force relogin


True

In [2]:
# Load data
translation_clusters = {}
idioms = set()
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

with open('dataset.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        idiom = row['english']
        translation = row['german']
        idioms.add(idiom)
        idioms.add(translation)

        if idiom in translation_clusters:
            translation_clusters[idiom].add(translation)
        else:
            translation_clusters[idiom] = {translation}

        if translation in translation_clusters:
            translation_clusters[translation].add(idiom)
        else:
            translation_clusters[translation] = {idiom}

# Convert idioms to tokenized representations
max_length = 0
encoder = {}

for idiom in idioms:
    encoded = tokenizer(idiom, return_tensors='pt', padding=True, truncation=True).input_ids
    max_length = max(max_length, encoded.shape[1])
    encoder[idiom] = encoded

num_idioms = len(idioms)
decoder = {}

# Generate idiom tensor
idiom_tensor = torch.zeros(size=(num_idioms, max_length), dtype=torch.long)
for i, idiom in enumerate(idioms):
    encoded = encoder[idiom]
    idiom_tensor[i, :encoded.shape[1]] = encoded[0]
    encoder[idiom] = idiom_tensor[i]
    decoder[tuple(idiom_tensor[i].tolist())] = idiom

assert decoder[tuple(encoder['makes me feel like'].tolist())] == 'makes me feel like'

# Convert idiom clusters to tokenized representations
translation_clusters_tokenized = {}
for idiom in translation_clusters:
    translation_clusters_tokenized[tuple(encoder[idiom].tolist())] = set()
    for match in translation_clusters[idiom]:
        if tuple(encoder[match].tolist()) == tuple(encoder[idiom].tolist()): continue
        translation_clusters_tokenized[tuple(encoder[idiom].tolist())].add(tuple(encoder[match].tolist()))

# Print the shape of idiom_tensor
print(idiom_tensor.shape)

translation_clusters = translation_clusters_tokenized


torch.Size([14673, 24])


In [3]:
train = idiom_tensor[:int(0.9*num_idioms)]
test = idiom_tensor[int(0.9*num_idioms):int(0.95*num_idioms)]
val = idiom_tensor[int(0.95*num_idioms)]

In [4]:
latent_dimensions = 64
device = 'mps'
iterations = 500
learning_rate = 0.00001
batch_size = 32
batch_accumulation = 1
roberta_output_length = 250002

wandb.init(
	project="Cross-Lingual-Idiom-Sense-Clustering",
	
	# track hyperparameters and run metadata
	config={
	"learning_rate": learning_rate,
	"architecture": "BERT",
	"batch_size": batch_size*batch_accumulation*2,
	"epochs": iterations,
	"embedding_dimensions":latent_dimensions,
	}
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: shayaan-absar. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
def tensor_to_set(tensor):
    return {tuple(d.tolist()) for d in tensor}

In [6]:
def create_batch(data):
	indexes = torch.randint(0, len(data), (batch_size,))
	batch = set()
	set_data = tensor_to_set(data)

	for i in indexes:
		idiom = data[i]

		possible_idioms = set_data.intersection(translation_clusters[tuple(idiom.tolist())])

		if len(possible_idioms) == 0:
			continue

		assert not(tuple(idiom.tolist()) in possible_idioms)

		random_cluster_mate = choice(list(possible_idioms))
		batch.add(tuple(random_cluster_mate))
		batch.add(tuple(idiom.tolist()))

	batch = torch.tensor([list(x) for x in list(batch)]).to(device)

	return batch

def get_batch(data):
	batch = torch.tensor([])

	while batch.numel() == 0:
		batch = create_batch(data)
	return batch

print(get_batch(train).shape)


torch.Size([62, 24])


In [9]:
def get_positive_sample(data):
    positive_samples = []

    for anchor in data:
        possible_positive = translation_clusters[tuple(anchor.tolist())].intersection(tensor_to_set(data))


        chosen = torch.tensor(choice(list(possible_positive)))
        positive_samples.append(chosen)

    positive_samples = torch.stack(positive_samples).to(device)

    return positive_samples


def get_negative_sample(data):
    negative_samples = []

    for anchor in data:
        possible_negative = tensor_to_set(data).difference(translation_clusters[tuple(anchor.tolist())])

        chosen = torch.tensor(choice(list(possible_negative)))
        negative_samples.append(chosen)

    negative_samples = torch.stack(negative_samples).to(device)

    return negative_samples

In [10]:
class Model(nn.Module):
	def __init__(self, pooling):
		super().__init__()
		self.pooling = pooling
		self.roberta = AutoModelForMaskedLM.from_pretrained('xlm-roberta-base')
		self.batch_norm = nn.BatchNorm1d(max_length)
		self.output_layer = nn.Linear(roberta_output_length, latent_dimensions)
		self.activation = nn.LeakyReLU()

	def forward(self, input):
		input_ids = torch.tensor(input, dtype=torch.long).to(device)
		attention_mask = torch.LongTensor(torch.ones(input.shape, dtype=torch.long)).to(device)
		roberta_logits = self.roberta(input_ids=input_ids, attention_mask=attention_mask).logits
		batch_norm_roberta_logits = self.batch_norm(roberta_logits)
		if self.pooling == 'mean': pooled = torch.mean(batch_norm_roberta_logits, dim=1)
		if self.pooling == 'max': pooled, _ = torch.max(batch_norm_roberta_logits, dim=1)
		if self.pooling == 'min': pooled, _ = torch.min(batch_norm_roberta_logits, dim=1)
		vector_representation = self.output_layer(pooled)
		activation = self.activation(vector_representation)

		return activation

In [43]:
model = Model(pooling='average')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [47]:
triplet_loss = nn.TripletMarginLoss()

def pass_batch(batch):
	torch.cuda.empty_cache()

	positive_samples = get_positive_sample(batch)
	negative_samples = get_negative_sample(batch)

	encodings = model(batch)
	positive_sample_encodings = model(positive_samples)
	negative_sample_encodings = model(negative_samples)

	return triplet_loss(encodings, positive_sample_encodings, negative_sample_encodings)

print(f'Batch size=~{batch_size*batch_accumulation*2}')
for _ in range(iterations):
	loss = 0
	for __ in range(batch_accumulation):
		batch = get_batch(train)
		loss += pass_batch(batch)
	loss /= batch_accumulation
	optimizer.zero_grad()
	loss.backward()
	optimizer.step()

	wandb.log({"loss": loss})
wandb.finish()

Batch size=~64


/tmp/ipykernel_32/1296365142.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input, dtype=torch.long).to(device)


loss,▁▅█▁▇▃▅█▅▃
loss,0.99436


In [11]:
run = wandb.init()
artifact = run.use_artifact('shayaan-absar/Cross-Lingual-Idiom-Sense-Clustering/model.pt:v0', type='model')
artifact_dir = artifact.download()
model = Model(pooling='mean').to(device)
model.load_state_dict(torch.load(os.path.join(artifact_dir, 'model.pt'), map_location=device))

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Downloading large artifact model.pt:v0, 1122.74MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:6.5


<All keys matched successfully>

In [55]:
batch_size=100
tds = create_batch(test)
print(tds.shape)

torch.Size([8, 24])


In [59]:
encodings = model(tds)
encodings = encodings.detach().cpu().numpy()
reducer = umap.UMAP(n_components=2)
encodings = reducer.fit_transform(encodings)

captions = [decoder[tuple(tds[i].tolist())] for i in range(tds.shape[0])]
count = 0
categories = [None for i in range(tds.shape[0])]

for i in range(tds.shape[0]):
    curr = tds[i]
    
    if categories[i] is not None:
        continue
        
    categories[i] = count
    
    for j in range(tds.shape[0]):
        if categories[j] is None:
            if tuple(tds[j].tolist()) in translation_clusters[tuple(curr.tolist())]:
                categories[j] = count
    
    count += 1

fig = px.scatter(x=encodings[:, 0], y=encodings[:, 1], color=categories, color_continuous_scale='Viridis')

fig.update_traces(marker=dict(size=15))

for i, caption in enumerate(captions):
    fig.add_annotation(
        x=encodings[i, 0],
        y=encodings[i, 1],
        text=caption,
        showarrow=True,
        arrowhead=2,
        font=dict(size=10),
    )

fig.update_layout(title="UMAP Projection on Fine-Tuned Model")
fig.show()

/var/folders/c9/ld6wd4rd4416tr7z2q29t7180000gn/T/ipykernel_82259/250995631.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/opt/homebrew/lib/python3.11/site-packages/umap/umap_.py:2344: UserWarning:

n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1



In [60]:
roberta = AutoModelForMaskedLM.from_pretrained('xlm-roberta-base').to(device)
encodings = roberta(tds).logits
encodings = torch.mean(encodings, dim=1)

encodings = encodings.detach().cpu().numpy()
reducer = umap.UMAP(n_components=2)
encodings = reducer.fit_transform(encodings)


fig = px.scatter(x=encodings[:, 0], y=encodings[:, 1], color=categories, color_continuous_scale='Viridis')
fig.update_traces(marker=dict(size=15))

for i, caption in enumerate(captions):
    fig.add_annotation(
        x=encodings[i, 0],
        y=encodings[i, 1],
        text=caption,
        showarrow=True,
        arrowhead=2,
        font=dict(size=10),
    )

fig.update_layout(title="UMAP Projection")
fig.show()


/opt/homebrew/lib/python3.11/site-packages/umap/umap_.py:2344: UserWarning:

n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1

